# 3. Import biosphere3 and ecoinvent

Now that you know how to work with the foreground system, it's time to learn how to work with the background system. In particular it is useful to import and play around with two databases: _biosphere3_ that contains all the exchanges and impact assessment methods, and _ecoinvent_. You need to get the ecoinvent files first, see steps below:

1. Open the ecoinvent website and [login](https://www.ecoinvent.org/login-databases.html) with your username and password.
2. You should read somewhere: _To download LCI and LCIA cumulative matrices click here_. Click there.
3. Select _ecoinvent 3.6._
4. Download the file `ecoinvent 3.6_consequential_ecoSpold02.7z` in a folder of yours. Make sure you remember the full path to this directory. E.g. I have downloaded the file in:
_/Users/massimo/Documents/Databases/ecoinvent v3.6_
5. The file you have downloaded is a compressed archive of many files (like with winzip or winrar). Extract the files from the .7z archive, e.g. by double clicking it. If it does not work, install a software that can do that. E.g. for mac users you can either download [theunarchiver](https://theunarchiver.com/) or, if you are using brew, just open terminal and do `brew install p7zip` and then from terminal find the folder and do `7z x 'ecoinvent 3.6_consequential_ecoSpold02.7z'` (here the [p7zip instructions](https://wiki.archlinux.org/index.php/p7zip) in case).
6. Now you an run the cells below. Make sure you change the path line and replace it with the one where you have extracted the files. For example, I have extracted the files in a folder called "datasets". The path to this folder is: _/Users/massimo/Documents/Databases/ecoinvent v3.6/datasets_ You will see this same line in the script and you need to change it with your directory. 

In [1]:
import brightway2 as bw
import os as os
#bw.databases.clear() # For a fresh start (Risky command! clears all your existing databases)

In [2]:
bw.projects # check what project you have 
if 'trial_bw2' in bw.projects:
    print('deleting project')
    bw.projects.delete_project('trial_bw2', delete_dir=True) # if you want a fresh start
bw.projects.set_current('trial_bw2') # Still working in the same project

In [3]:

bw.databases
bw.databases.clear() # For a fresh start (Risky command! clears all your existing databases)

Before importing ecoinvent, we need to make a default setup of Brightway2. This means importing all the environmental exchanges and all the LCIA methods. Then when we import ecoinvent the ecoinvent activities will be linked to this database of environmentla exchanges, just like in the previous example wiht the product system of H&S 2002.

In [4]:
# Import the biosphere3 database
bw.bw2setup() # This will take some time

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 08/07/2023 11:41:59
  Finished: 08/07/2023 11:42:00
  Total time elapsed: 00:00:00
  CPU %: 84.80
  Memory %: 2.35
Created database: biosphere3
Creating default LCIA methods

Wrote 762 LCIA methods with 227223 characterization factors
Creating core data migrations



We are going to use version 3.8 of ecoinvent, consequential model, for this course.

In [5]:
# Import ecoinvent
database_name = 'ecoinvent 3.8 conseq'
# You need to change the line below with the directory where you have saved ecoinvent
# ei36dir = "/Users/massimo/Documents/Databases/ecoinvent v3.6/datasets"
import os
if os.name == 'nt':
    eidir = os.path.join('/tools','databases','ecoinvent','ecoinvent 3.8','ecoinvent 3.8_consequential_ecoSpold02','datasets')
elif os.name == 'posix':
    eidir = os.path.join('/mnt','c','tools','databases','ecoinvent','ecoinvent 3.8','ecoinvent 3.8_consequential_ecoSpold02','datasets')


if database_name in bw.databases:
    print("Database has already been imported")
else:
    eidir = bw.SingleOutputEcospold2Importer(eidir, database_name) # You can give it another name of course
    eidir.apply_strategies()
    eidir.statistics()
    eidir.drop_unlinked(True)
    eidir.write_database() # This will take some time.

Extracting XML data from 17653 datasets
Extracted 17653 datasets in 137.48 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
11 exchanges couldn't be linked and were deleted. See the logfile for details:
	C:\Users\ypb23101\AppData\Local\pylca\Brightway3\Logs\trial_bw2.414f13

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:28


Title: Writing activities to SQLite3 database:
  Started: 08/07/2023 11:45:09
  Finished: 08/07/2023 11:45:37
  Total time elapsed: 00:00:28
  CPU %: 93.90
  Memory %: 30.16
Created database: ecoinvent 3.8 conseq


In [6]:
bw.databases # you should now see both "biosphere3" and "ecoinvent 3.6 conseq"

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.8 conseq

# Add Foreground Processes

In [7]:
# clean foreground

sssd_launch_name = "sssd_launch"
if sssd_launch_name in bw.databases:
    del bw.databases[sssd_launch_name]

sssd_launch_db = bw.Database(sssd_launch_name) # creates an instance of the database class # t_db.name for example.

In [8]:
# search checks
# bw.Database("biosphere3").search("carbon dioxide, fossil")
# Same but different:
for activity in bw.Database("biosphere3").search('carbon dioxide, fossil', limit = 5):  
    print(activity)
    print(activity['code'])
CO2 = bw.Database("biosphere3").get("349b29d1-3e58-4c66-98b9-9d1a076efd2e") # This code works across bw2 installations, 
                                                                    ### i.e. is univocal for biosphere3 everywhere
print(CO2['name']) # there is more than one activity with this name. Only code is univocal.
print(CO2['unit'])
print(CO2['database'])
print(CO2['code'])

'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term'))
e259263c-d1f1-449f-bb9b-73c6d0a32a00
'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))
aa7cac3a-3625-41d4-bc54-33e2cf11ec46
'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
16eeda8a-1ea2-408e-ab37-2648495058dd
'Carbon dioxide, fossil' (kilogram, None, ('air',))
349b29d1-3e58-4c66-98b9-9d1a076efd2e
'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))
f9749677-9c9f-4678-ab55-c607dfdc2cb9
Carbon dioxide, fossil
kilogram
biosphere3
349b29d1-3e58-4c66-98b9-9d1a076efd2e


In [9]:
for activity in bw.Database("biosphere3").search('carbon monoxide, fossil', limit = 10):  
    print(activity)
    print(activity['code'])


'Carbon monoxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
a3beb5ac-5149-47f6-a035-53fc5030f10a
'Carbon monoxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))
099b36ab-4c03-4587-87f4-2f81e337afb8
'Carbon monoxide, fossil' (kilogram, None, ('air',))
ba2f3f82-c93a-47a5-822a-37ec97495275
'Carbon monoxide, fossil' (kilogram, None, ('air', 'low population density, long-term'))
49c594c6-b6a8-42a3-95c5-cca812fda80b
'Carbon monoxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))
6edcc2df-88a3-48e1-83d8-ffc38d31c35b
'Carbon monoxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))
b821ad93-ba6a-4d74-b6ae-ca12cd9563d7
'Carbon monoxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground'))
9afa0173-ecbd-4f2c-9c5c-b3128a032812
'Carbon monoxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
968495ec-1000-438a-9e11-467776ebb54e
'Carbon monoxide, non-

In [10]:
# bw.Database("biosphere3").search("Sulfur dioxide")
for activity in bw.Database("biosphere3").search('Sulfur dioxide', filter={"categories" : 'air'},limit = 5):  
    print(activity)
    print(activity['code'])
SO2 = bw.Database("biosphere3").get("fd7aa71c-508c-480d-81a6-8052aad92646") # This code works across bw2 installations, 
                                                                    ### i.e. is univocal for biosphere3 everywhere
print(SO2['name']) # there is more than one activity with this name. Only code is univocal.
print(SO2['unit'])
print(SO2['code'])

Excluding 0 filtered results
'Sulfur dioxide' (kilogram, None, ('air', 'non-urban air or from high stacks'))
78c3efe4-421c-4d30-82e4-b97ac5124993
'Sulfur dioxide' (kilogram, None, ('air', 'low population density, long-term'))
6ce44f77-d181-4396-8fa2-2276eeeb4c08
'Sulfur dioxide' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
16521d00-4b6f-464f-b5dd-8da4a27b15ca
'Sulfur dioxide' (kilogram, None, ('air',))
fd7aa71c-508c-480d-81a6-8052aad92646
'Sulfur dioxide' (kilogram, None, ('air', 'urban air close to ground'))
8c52f40c-69b7-4538-8923-b371523c71f5
Sulfur dioxide
kilogram
fd7aa71c-508c-480d-81a6-8052aad92646


In [11]:
# bw.Database("biosphere3").search("Sulfur dioxide")
for activity in bw.Database("biosphere3").search('nitrogen oxides',filter={"categories" : 'air'},limit = 5):  
    print(activity)
    print(activity['code'])
NOx = bw.Database("biosphere3").get("c1b91234-6f24-417b-8309-46111d09c457") # This code works across bw2 installations, 

#NOx = bw.Database("biosphere3").get('fa5512cf-04a2-4014-8e96-d3e2d137f0df') # This code works across bw2 installations, 
                                                                    ### i.e. is univocal for biosphere3 everywhere
print(NOx['name']) # there is more than one activity with this name. Only code is univocal.
print(NOx['unit'])
print(NOx['code'])

Excluding 0 filtered results
'Nitrogen oxides' (kilogram, None, ('air', 'urban air close to ground'))
d068f3e2-b033-417b-a359-ca4f25da9731
'Nitrogen oxides' (kilogram, None, ('air',))
c1b91234-6f24-417b-8309-46111d09c457
'Nitrogen oxides' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
4841a0fe-c250-4a39-94a1-1bc31426c0f1
'Nitrogen oxides' (kilogram, None, ('air', 'non-urban air or from high stacks'))
77357947-ccc5-438e-9996-95e65e1e1bce
'Nitrogen oxides' (kilogram, None, ('air', 'low population density, long-term'))
9115356e-a534-4329-9ec6-d9208720241b
Nitrogen oxides
kilogram
c1b91234-6f24-417b-8309-46111d09c457


In [12]:
# This is the most important cell in this notebook, read it carefully
sssd_launch_db.write({
    (sssd_launch_name, "Electricity production"):{ # Note that a tuple is used to identify an activity univocally
        'name':'Electricity production',
        'unit': 'kWh', 
        'exchanges': [{
                'input': (sssd_launch_name, 'Fuel production'),
                'amount': 2,
                'unit': 'liters',
                'type': 'technosphere'
            },{
                'input': ('biosphere3', CO2['code']),
                'amount': 1,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('biosphere3', SO2['code']),
                'amount': 0.1,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('biosphere3', NOx['code']),
                'amount': 0.2,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': (sssd_launch_name, 'Electricity production'), #important to write the same process name in output
                'amount': 10,
                'unit': 'kWh',
                'type': 'production'
            }]
        },
    (sssd_launch_name, 'Fuel production'):{ # here starts another activity
        'name': 'Fuel production',
        'unit': 'liters',
        'exchanges':[{
                'input': ('biosphere3', CO2['code']),
                'amount': 30,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('biosphere3', SO2['code']),
                'amount': 2,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': ('biosphere3', NOx['code']),
                'amount': 4,
                'unit': 'kg',
                'type': 'biosphere'
            },{
                'input': (sssd_launch_name, 'Crude oil'),
                'amount': -50,
                'unit': 'liters',
                'type': 'biosphere'
            },{
                'input': (sssd_launch_name, 'Fuel production'),
                'amount': 100,
                'unit': 'liters',
                'type': 'production'
            }]
    },
    #(sssd_launch_name, 'Carbon dioxide'):{'name': 'Carbon dioxide', 'unit':'kg', 'type': 'biosphere'}, # env exchanges
    #(sssd_launch_name, 'Sulphur dioxide'):{'name': 'Sulphur dioxide', 'unit':'kg', 'type': 'biosphere'},
    (sssd_launch_name, 'Crude oil'):{'name': 'Crude oil', 'unit':'liters', 'type': 'biosphere'}

    })

Writing activities to SQLite3 database:
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 08/07/2023 11:47:37
  Finished: 08/07/2023 11:47:37
  Total time elapsed: 00:00:00
  CPU %: 104.20
  Memory %: 24.28


# Calculate with biosphere3 and ecoinvent 

Ok now we can run an LCA with a dataset from ecoinvent.

In [13]:
list(bw.methods)

[('CML v4.8 2016 no LT',
  'acidification no LT',
  'acidification (incl. fate, average Europe total, A&B) no LT'),
 ('CML v4.8 2016 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: freshwater no LT',
  'freshwater aquatic ecotoxicity (FAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: marine no LT',
  'marine aquatic ecotoxicity (MAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: terrestrial no LT',
  'terrestrial ecotoxicity (TETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'energy resources: non-renewable no LT',
  'abiotic depletion potential (ADP): fossil fuels no LT'),
 ('CML v4.8 2016 no LT',
  'eutrophication no LT',
  'eutrophication (fate not incl.) no LT'),
 ('CML v4.8 2016 no LT',
  'human toxicity no LT',
  'human toxicity (HTP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'material resources: metals/minerals no LT',
  'abiotic depletion potential (ADP): elements (ultimate reserves) no LT')

In [14]:
# First select a method from the list (use 'list(methods)' to see all of them)
# mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
method_key =  ('IPCC 2021', 'climate change: land use, including SLCFs', 'global warming potential (GWP100)')
method_key =  ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
method_key =  ('IPCC 2021', 'climate change: including SLCFs', 'global warming potential (GWP100)')
method_key
LCIAdata_t = bw.Method(method_key).load() 
print(LCIAdata_t)
print(type(LCIAdata_t))
LCIAdata = []
for ii in LCIAdata_t:
    LCIAdata.append([ii[0],ii[1]])
print(LCIAdata)
myLCIAdata=[[('biosphere3', NOx['code']), 5], [('biosphere3', SO2['code']), -1]]

LCIAdata.extend(myLCIAdata)
print(LCIAdata)

my_method_key = ('IPCC 2021 mod', 'climate change: including SLCFs', 'global warming potential (GWP100)')
my_method = bw.Method(my_method_key)
my_method.validate(LCIAdata)
my_method.register() 
my_method.write(LCIAdata)
my_method.load()

[(('biosphere3', '892f10da-7bf2-42b3-b171-221989661a05'), 0.006), (('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'), 1), (('biosphere3', '6edcc2df-88a3-48e1-83d8-ffc38d31c35b'), 4.0624), (('biosphere3', '9afa0173-ecbd-4f2c-9c5c-b3128a032812'), 2.491), (('biosphere3', '4cadbc9f-0472-4bb9-a942-df5b90e37878'), 20.6), (('biosphere3', '9ea11aea-7cb7-4b38-b936-a42c2a4bac97'), 0.437), (('biosphere3', 'fa0c2bee-8dd9-4f8a-8489-b1f3b43de958'), 1530), (('biosphere3', '818cee9e-231c-4b53-8ed2-47a0001802d5'), 161), (('biosphere3', 'f43b2b80-f5a6-4d31-9980-0754305a978d'), 5810), (('biosphere3', '033d3a16-e1a2-4ce8-8769-f4ea867801ca'), 860), (('biosphere3', '393d0862-04cc-45ae-8d08-9dc2a4461b35'), 164), (('biosphere3', '468bebea-17ee-49d3-b11d-c603d91c67c8'), 1.3), (('biosphere3', '400be66e-4383-482a-93e3-c005e96de261'), 2300), (('biosphere3', 'db907ff7-6cf0-4f88-9376-116262dfad6d'), 90.4), (('biosphere3', '27f61651-96a5-45c7-9873-832626cf6905'), 597), (('biosphere3', '15e54d5b-aba2-40a6-9aa8-a

[[('biosphere3', '892f10da-7bf2-42b3-b171-221989661a05'), 0.006],
 [('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'), 1],
 [('biosphere3', '6edcc2df-88a3-48e1-83d8-ffc38d31c35b'), 4.0624],
 [('biosphere3', '9afa0173-ecbd-4f2c-9c5c-b3128a032812'), 2.491],
 [('biosphere3', '4cadbc9f-0472-4bb9-a942-df5b90e37878'), 20.6],
 [('biosphere3', '9ea11aea-7cb7-4b38-b936-a42c2a4bac97'), 0.437],
 [('biosphere3', 'fa0c2bee-8dd9-4f8a-8489-b1f3b43de958'), 1530],
 [('biosphere3', '818cee9e-231c-4b53-8ed2-47a0001802d5'), 161],
 [('biosphere3', 'f43b2b80-f5a6-4d31-9980-0754305a978d'), 5810],
 [('biosphere3', '033d3a16-e1a2-4ce8-8769-f4ea867801ca'), 860],
 [('biosphere3', '393d0862-04cc-45ae-8d08-9dc2a4461b35'), 164],
 [('biosphere3', '468bebea-17ee-49d3-b11d-c603d91c67c8'), 1.3],
 [('biosphere3', '400be66e-4383-482a-93e3-c005e96de261'), 2300],
 [('biosphere3', 'db907ff7-6cf0-4f88-9376-116262dfad6d'), 90.4],
 [('biosphere3', '27f61651-96a5-45c7-9873-832626cf6905'), 597],
 [('biosphere3', '15e54d5b-a

In [15]:
el = sssd_launch_db.get('Electricity production')  # reads: "get the activity electricity production from t_db 
                                         # and call it el"
print(el)

'Electricity production' (kWh, None, None)


In [16]:
functional_unit = {el : 1}
lca = bw.LCA(functional_unit, my_method_key) #run LCA calculations again with method
lca.lci()
lca.lcia()
print(lca.score) # What is the unit? Find out! (alright it's bw.methods[mymethod])

0.2860000013411045
